In [ ]:
scaler = StandardScaler().fit(features)
X_train = scaler.transform(features)

pca = PCA(n_components=25)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
x_train = X_train_pca[:,0:3] # Three best features

plt.figure(figsize=(8,6))
plt.scatter(x_train[:,0], x_train[:,1], c=y)

In [ ]:
# perform pca on features
plt.bar(range(0,25), pca.explained_variance_ratio_, label="individual var");
plt.step(range(0,25), np.cumsum(pca.explained_variance_ratio_),'r', label="cumulative var");
plt.xlabel('Principal component index'); plt.ylabel('explained variance ratio %');
plt.legend()

In [ ]:
# Information gain
from sklearn.feature_selection import mutual_info_classif
%matplotlib inline

scaler = StandardScaler().fit(features)
X_train = scaler.transform(features)

importances = mutual_info_classif(X_train, y)
feat_importances = pd.Series(importances, features.columns[0:len(features.columns)])
feat_importances.plot(kind='bar', color='teal')
plt.show()

In [ ]:
total_features_index = []

for a in total_features:
    total_features_index.append(a)

print(total_features_index)
print(len(total_features_index))


In [ ]:
# Variance Threshold

from sklearn.feature_selection import VarianceThreshold

scaler = StandardScaler().fit(total_features)
X_train = scaler.transform(total_features)
X = X_train[0:,0:49]

v_threshold = VarianceThreshold(threshold=0)
v_threshold.fit(X)
index = v_threshold.get_support()
true_index = [i for i, x in enumerate(index) if x]

# for i in true_index:
#     print(total_features_index[i])
#     print(total_features[total_features_index[i]])
vt_features = [total_features_index[i] for i in true_index]
features = total_features[vt_features]
# features.head()




In [ ]:
#Pearson Correlation
import seaborn as sns

# total_features = pd.merge(edafeatures, ecgfeatures, left_index=True, right_index=True)
out_features = total_features
out_features['y'] = y
# plt.figure(figsize=(12,10))
cor = out_features.corr()
# print(cor)
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()
#Correlation with output variable
cor_target = abs(cor["y"])

# print(cor_target)
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.3]
# features = out_features[relevant_features.index[:-1]]
relevant_features

In [ ]:
# HR_mean, scr_mobility scr_integral 
print(out_features[['HR_mean',"scr_mobility"]].corr())
print(out_features[["HR_mean","scr_integral"]].corr())
print(out_features[["scr_mobility","scr_integral"]].corr())
# features = features[['HR_mean',"scr_mobility","scr_integral"]]
features = out_features[["HR_mean", "scr_mobility", "scr_integral"]]

In [ ]:
# # Forward wrapper

# from mlxtend.feature_selection import SequentialFeatureSelector
# ffs = SequentialFeatureSelector(total_features, k_features='best', forward = 'True', n_jobs=-1)
# ffs.fit(X, y)
# features= list(ffs.k_feature_names_)
# features= list(map(int, features))
# lr.fit[x_train[features], y_train]
# y_pred = lr.predict(x_train[features])

In [ ]:
# Backwards
import statsmodels.api as sm

cols = list(total_features.columns)
pmax = 1

while (len(cols)>0):
    p= []
    X_1 = total_features[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
features = total_features[selected_features_BE]

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
X = features

In [ ]:
#Wrapper Forwards
sfs = SFS(svm.SVC(),
          k_features=6,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)
sfs.fit(X, y)
feature_namesf = list(sfs.k_feature_names_)

In [ ]:
#Wrapper Backwards
sbs = SFS(svm.SVC(),
         k_features=6,
         forward=False,
         floating=False,
         cv=0)
sbs.fit(X, y)
feature_namesb = list(sbs.k_feature_names_)

In [ ]:
#Step-wise wrapper
sffs = SFS(svm.SVC(),
         k_features=(3,11),
         forward=True,
         floating=True,
         cv=0)
sffs.fit(X, y)
feature_namess = list(sffs.k_feature_names_)

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()

In [ ]:
features = total_features[feature_namess]

features.head()

In [ ]:
# Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold

X = features
alphas = np.logspace(-4, -0.5, 5)

out_k_fold = KFold(3, shuffle=False)
in_k_fold = KFold(3, shuffle=False)
for (calib, test) in out_k_fold.split(X, y):
    X_calib, y_calib = X[calib[0]:calib[-1]], y[calib[0]:calib[-1]]
    X_test, y_test = X[test[0]:test[-1]], y[test[0]:test[-1]]
    lasso_cv = LassoCV(alphas=alphas, random_state=0, max_iter=10000, cv=in_k_fold)
    lasso_cv.fit(X_calib, y_calib)
    print(lasso_cv.alpha_, lasso_cv.score(X_test, y_test))
    break

In [ ]:
from sklearn.feature_selection import SelectFromModel

sel_ = SelectFromModel(
    LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=10))
sel_.fit(scaler.transform(X_train), y_train)

a =(sel_.estimator_.coef_ == 0).ravel().tolist()

# removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
X_train_selected = sel_.transform(scaler.transform(X_train))
X_test_selected = sel_.transform(scaler.transform(X_test))
# selected_columns_lasso = logistic.columns[selected_featuress.var() != 0]

# Y_test_pred = logistic_model.predict(X_test)
# log_score = logistic_model.score(X_test,y_test)